### Setup Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
print("Version: ", tf.__version__)

### Basic Neuron

In [ ]:
# To simulate XOR Neuron with input [[0.0,0.0], [0.0,1.0], [1.0,0.0], [1.0,1.0]] and output [[0],[1],[1],[0]]
X = [tf.constant([[0.0,0.0]]),
     tf.constant([[0.0,1.0]]),
     tf.constant([[1.0,0.0]]),
     tf.constant([[1.0,1.0]])]

In [ ]:
# Layer 1: Create 2 X 3 Weight (W1) and 1 X 3 bias (b1)
W1 = tf.constant([[-1.1, 3.3, 2.8],
                 [-1.0, 3.3, 2.8]])
b1 = tf.constant([[ 0.0, 0.0, -2.8]])

In [ ]:
# Layer 2: Create 3 X 1 Weight (W2) and 1 X 1 bias (b2)
W2 = tf.constant([[ 0.5],
                  [ 2.9 ],
                  [-6.8]])
b2 = tf.constant([[-4.6]])

In [ ]:
for i in range(4):
    a1 = tf.matmul(X[i],W1) + b1
    z1 = keras.activations.relu(a1)
    a2 = tf.matmul(z1,W2) + b2
    Y = keras.activations.sigmoid(a2)
    print(X[i].numpy(),Y.numpy())

### How to train XOR using NN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the XOR input and output data
X = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
y = np.array([[0.0], [1.0], [1.0], [0.0]])

# Build the neural network model
model = Sequential()
model.add(Dense(3,input_dim=2,activation='relu'))  # Hidden layer with 3 neuron, input_dim=2, activation='relu'
model.add(Dense(1,activation='sigmoid'))            # Output layer with 1 neuron, activation='sigmoid'

In [ ]:
model.summary()

In [ ]:
# Compile the model
ADAM = tf.keras.optimizers.Adam(learning_rate=0.02) # learning_rate=0.02
model.compile(optimizer=ADAM, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
EPOCHS = 200 # EPOCHS = 200
model.fit(X, y, epochs=EPOCHS)

In [ ]:
model.predict(X)

## Cats and Dogs Small dataset


#### Step 1: Import the Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

# For PC
# PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

# For Google Colab
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_extracted','cats_and_dogs_filtered')

PATH

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [ ]:
batch_size = 128
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
# Data Augmentation
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )
image_gen_val = ImageDataGenerator(rescale=1./255)

In [ ]:
# Batch Process
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='categorical')

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='categorical')

##### Visualize the Data

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

#### Step 2: Define  the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model_batch_dropout_aug = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape = (IMG_HEIGHT, IMG_WIDTH,3)),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    # Drop Out layer
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])

#### Step 3: Train the Model

In [ ]:
ADAM = tf.keras.optimizers.Adam(learning_rate=0.0005)
model_batch_dropout_aug.compile(optimizer=ADAM,loss='categorical_crossentropy',metrics=['accuracy'])

EPOCHS = 55
history = model_batch_dropout_aug.fit(train_data_gen,epochs=EPOCHS,validation_data=val_data_gen)

#### Step 4: Evaluate the Model

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epoch = range(len(loss))

import matplotlib.pyplot as plt

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epoch,loss,label='loss')
plt.plot(epoch,val_loss,label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch,acc,label='acc')
plt.plot(epoch,val_acc,label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

#### Step 5: Test the Model

In [ ]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

img_path=['cat.jpg','dog.jpg','test_cat.jpg','test_dog.jpg']

for i in range(4):
    img = image.load_img(img_path[i], target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    preds = model_batch_dropout_aug.predict(img_tensor)
    output = np.argmax(preds)
    label = 'cat' if output == 0 else 'dog'

    plt.imshow(img_tensor[0])
    plt.show()
    print(preds)
    print('The model prediction is ',label)

#### Step 6: Save and Convert to tflite Model

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model_batch_dropout_aug)
tflite_model = converter.convert()

# Save the model.
with open('cat_dog_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!pip install tflite-support-nightly

In [ ]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

In [ ]:
ImageClassifierWriter = image_classifier.MetadataWriter
_MODEL_PATH = "cat_dog_model.tflite"
# Task Library expects label files that are in the same format as the one below.
_LABEL_FILE = "cat_dog_labels.txt"
_SAVE_TO_PATH = "cat_dog_metadata.tflite"
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# End of Worksheet